In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('omw-1.4')
import os
import re

[nltk_data] Downloading package wordnet to /Users/m1mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/m1mac/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/m1mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = stopwords.words('english')

In [8]:
#This function removes stop words and lemmatizes the words.
def stop_word_and_lemmatize(temp, stopwords):
    with open(temp, 'r+') as temp:
        temp.seek(0)
        tempstring = temp.read()
        words = tempstring.split()
        lemmatizer = WordNetLemmatizer()
        temp.seek(0)
        temp.truncate(0)
        for r in words:
            temp2 = re.sub('[^a-zA-Z0-9 \n\.]','', r).lower()
            if temp2 not in stop_words:
                temp.write(""+lemmatizer.lemmatize(temp2)+" ")

In [9]:
#opens all the file in the directory.
for file in os.listdir():
    if file.endswith(".txt"):
        stop_word_and_lemmatize(file, stopwords)

In [10]:
#creates a list of file names
file_names = [];
for file in os.listdir():
    if(file.endswith(".txt")):
        file_names.append(file)

In [11]:
#generates postings of all the words.
postings = {};
for file_num in range(len(file_names)):
    f = open(file_names[file_num], "rb")
    words = f.read().decode(errors="ignore").split()
    for word in words:
        if word in postings:
            postings[word].append(file_num)
        else:
            postings[word] = [file_num]
            
for x in postings:
    postings[x] = list(set(postings[x]))

In [12]:
postings

{'much': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  41],
 'ado': [0, 2, 3, 35, 6, 8, 41, 13, 14, 17, 20, 22, 31],
 'nothing': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  41],
 'william': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41],
 'shakespeare': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,

In [13]:
#list of all the numbers corresponding to files for purpose of not operation.
all_postings = [];
for i in range(len(file_names)):
    all_postings.append(i)

In [14]:
#finds edit distance between two words
def edit_distance(word1, word2):
    m = len(word1)
    n = len(word2)
    arr = [[0 for x in range(n + 1)] for x in range(m + 1)]
    for i in range(m+1):
        for j in range(n+1):
            if i==0:
                arr[i][j] = j
            elif j==0:
                arr[i][j] = i
            elif word1[i-1]==word2[j-1]:
                arr[i][j] = arr[i-1][j-1]
            else:
                arr[i][j] = min(arr[i-1][j-1], arr[i-1][j], arr[i][j-1])+1
            
    return arr[m][n];

In [63]:
#returns the list of file names corresponding to a boolean query
def querying_normal(query):
    allwords = query.split()
    querywords = []
    queryops = []
    for i in allwords:
        if i=="AND" or i=="OR":
            queryops.append(i)
        else:
            querywords.append(i)

    flag=1
    recommonded = {}
    for word in querywords:
        if word[0]=='!':
            word = word[1:]
        if word not in postings:
            edit_distances = {}
            flag = 0
            for word2 in postings:
                temp = edit_distance(word, word2)
                if temp<=0.35*len(word):
                    edit_distances[word2] = temp
            recommonded[word] = [edit_distances.keys()]
        
    

    if flag==1:
        if querywords[0][0]!='!':
            final_answer = postings[querywords.pop(0)].copy()
        else:
            temp_word = querywords.pop(0)[1:]
            final_answer = [item for item in all_postings if item not in postings[temp_word]]
        for i in queryops:
            if i=="AND":
                if querywords[0][0] != "!":
                    first = postings[querywords.pop(0)].copy()
                    final_answer = list(set(final_answer) & set(first))
                else:
                    temp_word = querywords.pop(0)[1:]
                    final_answer = [item for item in all_postings if item not in postings[temp_word]].copy()
                    final_answer = list(set(final_answer))
            if i=='OR':
                if querywords[0][0] != "!":
                    first = postings[querywords.pop(0)].copy()
                    final_answer = list(set(final_answer) | set(first))
                else:
                    temp_word = querywords.pop(0)[1]
                    first = [item for item in all_postings if item not in postings[temp_word]].copy()
                    final_answer = list(set(final_answer) | set(first))
        for i in range(len(final_answer)):
            final_answer[i] = file_names[final_answer[i]] 
        return final_answer
    else:
        final_answer = ["Nothing found"]
        for word in recommonded:
            x = "Try"
            tempflag = 0;
            for word2 in recommonded[word]:
                x = x+str(word2)[9:]
                tempflag=1
            x = x+"instead of "+word
        if tempflag==1:
            final_answer.append(x)
    return final_answer

In [61]:
#takes an input string as query
query = input("Enter boolean query: ")

Enter boolean query: bubble OR imperfect


In [62]:
#applying querying_normal on query.
querying_normal(query)

10

In [26]:
#generates all rotations of the word so as to generate permuterms.
def generate_all_rotations(word):
    word = word+'$'
    n=len(word)
    word = word+word
    ans = []
    for i in range(n):
        temp = ""
        for j in range(n):
            temp = temp+word[i+j]
        ans.append(temp)
        
    return ans

In [27]:
#testing the function on a sample string
generate_all_rotations("hello")

['hello$', 'ello$h', 'llo$he', 'lo$hel', 'o$hell', '$hello']

In [28]:
#generates all the wild card postings
wild_card_postings = {}
for word in postings:
    temp_rotations = generate_all_rotations(word)
    for word2 in temp_rotations:
        if word2 not in wild_card_postings:
            wild_card_postings[word2] = word

In [29]:
wild_card_postings

{'much$': 'much',
 'uch$m': 'much',
 'ch$mu': 'much',
 'h$muc': 'much',
 '$much': 'much',
 'ado$': 'ado',
 'do$a': 'ado',
 'o$ad': 'ado',
 '$ado': 'ado',
 'nothing$': 'nothing',
 'othing$n': 'nothing',
 'thing$no': 'nothing',
 'hing$not': 'nothing',
 'ing$noth': 'nothing',
 'ng$nothi': 'nothing',
 'g$nothin': 'nothing',
 '$nothing': 'nothing',
 'william$': 'william',
 'illiam$w': 'william',
 'lliam$wi': 'william',
 'liam$wil': 'william',
 'iam$will': 'william',
 'am$willi': 'william',
 'm$willia': 'william',
 '$william': 'william',
 'shakespeare$': 'shakespeare',
 'hakespeare$s': 'shakespeare',
 'akespeare$sh': 'shakespeare',
 'kespeare$sha': 'shakespeare',
 'espeare$shak': 'shakespeare',
 'speare$shake': 'shakespeare',
 'peare$shakes': 'shakespeare',
 'eare$shakesp': 'shakespeare',
 'are$shakespe': 'shakespeare',
 're$shakespea': 'shakespeare',
 'e$shakespear': 'shakespeare',
 '$shakespeare': 'shakespeare',
 'edited$': 'edited',
 'dited$e': 'edited',
 'ited$ed': 'edited',
 'ted$edi': 

In [44]:
#returns all the file names in which a particular wild card query is present.
def wild_card_query(query):
    final_answer =[]
    if query[0]=='*' and query[len(query)-1]=='*':
        query = query[1:len(query)-1]+'*'
        for word in wild_card_postings:
            if word[:len(query)-1]==query[:len(query)-1]:
                word2 =wild_card_postings[word]
                for word3 in postings[word2]:
                    final_answer.append(word3)
    elif query[len(query)-1]=='*':
        for word in wild_card_postings:
            if word[:len(query)-1]==query[:len(query)-1] and word.endswith('$'):
                word2 = wild_card_postings[word]
                for word3 in postings[word2]:
                    final_answer.append(word3)
            
    elif query[0] == '*':
        query = query[1:]+'$'
        for word in wild_card_postings:
            if word[:len(query)]==query:
                word2 = wild_card_postings[word]
                for word3 in postings[word2]:
                    final_answer.append(word3)
    else:
        i=0
        while query[i]!='*':
            i=i+1
        query = query[i+1:]+'$'+query[0:i]+'*'
        
        
    for i in range(len(final_answer)):
        final_answer[i] = file_names[final_answer[i]]
    return set(final_answer)

In [45]:
query2 = input("Enter any wild card query: ")

Enter any wild card query: hap*


In [46]:
wild_card_query(query2)

{'a-midsummer-nights-dream_TXT_FolgerShakespeare.txt',
 'alls-well-that-ends-well_TXT_FolgerShakespeare.txt',
 'antony-and-cleopatra_TXT_FolgerShakespeare.txt',
 'as-you-like-it_TXT_FolgerShakespeare.txt',
 'coriolanus_TXT_FolgerShakespeare.txt',
 'cymbeline_TXT_FolgerShakespeare.txt',
 'hamlet_TXT_FolgerShakespeare.txt',
 'henry-iv-part-1_TXT_FolgerShakespeare.txt',
 'henry-iv-part-2_TXT_FolgerShakespeare.txt',
 'henry-v_TXT_FolgerShakespeare.txt',
 'henry-vi-part-1_TXT_FolgerShakespeare.txt',
 'henry-vi-part-2_TXT_FolgerShakespeare.txt',
 'henry-vi-part-3_TXT_FolgerShakespeare.txt',
 'henry-viii_TXT_FolgerShakespeare.txt',
 'julius-caesar_TXT_FolgerShakespeare.txt',
 'king-john_TXT_FolgerShakespeare.txt',
 'king-lear_TXT_FolgerShakespeare.txt',
 'loves-labors-lost_TXT_FolgerShakespeare.txt',
 'lucrece_TXT_FolgerShakespeare.txt',
 'macbeth_TXT_FolgerShakespeare.txt',
 'measure-for-measure_TXT_FolgerShakespeare.txt',
 'much-ado-about-nothing_TXT_FolgerShakespeare.txt',
 'othello_TXT_Fo